<a href="https://colab.research.google.com/github/vkthakur90/jupyter_notebooks/blob/master/Pretrained_ConvAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.models as tfkm
import tensorflow.keras.layers as tfkl
import tensorflow.keras.utils as tfku

In [3]:
(x_train, y_train), (x_test, y_test) = tfk.datasets.cifar10.load_data()

In [4]:
def create_model():
    input_images = tfkl.Input([32, 32, 3])
    input_labels = tfkl.Input([2])

    base_model = tfk.applications.InceptionResNetV2(include_top = False, input_shape = [75, 75, 3], weights = 'imagenet', pooling = None)
    base_model.trainable = False

    x = input_images
    x = tfkl.Lambda(lambda t: tf.image.resize_with_pad(t, target_height = 75, target_width = 75))(x)
    x = tfkl.Lambda(lambda t: tfk.applications.inception_resnet_v2.preprocess_input(t))(x)
    x = base_model(x)
    x = tfkl.Reshape([-1, 1536])(x)

    y = tfkl.Embedding(input_dim = 2, output_dim = 1536)(input_labels)

    res = y
    y = tfkl.Attention(use_scale = True)([y, x])
    y = tfkl.Add()([y, res])

    y = tfkl.Flatten()(y)
    y = tfkl.Dense(10, activation = 'softmax')(y)
    out = y

    model = tfkm.Model(inputs = [input_images, input_labels], outputs = [out])

    model.compile(
        optimizer = 'adam',
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )

    return model

In [5]:
model = create_model()

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 lambda (Lambda)             (None, 75, 75, 3)            0         ['input_1[0][0]']             
                                                                                                  
 lambda_1 (Lambda)           (None, 75, 75, 3)            0         ['lambda[0][0]']              
                                                                                                  
 input_2 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                              

In [7]:
train_label = np.concatenate([0 * np.ones_like(y_train), np.ones_like(y_train)], axis = -1)

In [8]:
model.fit(x = [x_train, train_label], y = y_train, epochs = 1, batch_size = 8, shuffle = True)

6250/6250 [==============================] - 184s 26ms/step - loss: 1.8051 - accuracy: 0.6551


In [9]:
test_label = np.concatenate([0 * np.ones_like(y_test), np.ones_like(y_test)], axis = -1)

In [10]:
model.evaluate(x = [x_test, test_label], y = y_test)

313/313 [==============================] - 21s 53ms/step - loss: 1.6833 - accuracy: 0.6868


[1.68329918384552, 0.6868000030517578]